In [1]:
from logisticRegression import LogisticRegression, Optimizer
from scipy.io.arff import loadarff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def addInteractions(X):
    newX=X
    for i in range(X.shape[1]):
        for j in range(i+1,X.shape[1]):
            newX=np.c_[newX,np.multiply(X[:,i],X[:,j])]
    return newX

1. Add a stopping rule
2. Perform tests for max 500 iterations (unless it converges earlier), use balanced accuracy (so do some train test splits, at least 5)
3. Check how the log likelihood value depends on iterations for each algorithm for train data
4. Compare the algorithms with 4 other existing solutions such as: LDA (Linear Discriminant analysis), QDA (Quadratic
Discriminant Analysis), Decision tree and Random Forest
5. In the case of small datasets, please compare the two versions of the logistic regression: model
without interactions and model with interactions.

1. Stopping rule: If the differences in loss function are smaller than 0.0001, stop.

In [ ]:
# This is just an overall idea, feel free to delete it all or modify or whatever :)
# I'm wondering if this way of keeping results is optimal for later generating graphs from them
# Currently it's just the result, name of the classifier, and split, seed
# hmm cause for the accuracies we will probably want to draw boxplots, so like on the y axis accuracy, on the x axis train test split and then for each train test split a boxplot for each type of classifier
# I guess it should be possible with sns, maybe by setting y to accuracy, x to split and hue by classifier type or smth like that
# For the plots of loss function, I'm not sure how to represent it, whether to compute the mean for each iteration and plot a line going through those points
# or to just take a single result and plot that
def performExperiment(X,y,logisticparams):
    costs=[]
    accuracies=[]
    splits = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    seeds = [42,123,0,321,9]
    classifiersNames = ['Linear Discriminant Analysis','Quadratic Discriminant Analysis', 'Decision Tree Classifier', 'Random Forest Classifier']
    classifiers = [LinearDiscriminantAnalysis(),QuadraticDiscriminantAnalysis(),DecisionTreeClassifier(random_state=seed),RandomForestClassifier(random_state=seed)]
    for split in splits:
        for seed in seeds:
            np.random.seed(seed)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=seed)

            for logisticparam in len(logisticparams):
                logisticRegressor = LogisticRegression(learningRate=logisticparam.learningRate,noOfIterations=500, optimizer=logisticparam.optimizer)
                costs = logisticRegressor.fit(X_train.astype(float),y_train.astype(float))
                y_pred=logisticRegressor.predict(X_test)
                costs.append([costs,logisticparam.optimizer,seed,split])
                accuracies.append([1-np.sum(np.abs(y_pred-y_test))/len(y_test),logisticparam.optimizer,seed,split])

            for i,classifier in enumerate(classifiers):
                costs = classifier.fit(X_train.astype(float),y_train.astype(float))
                y_pred=classifier.predict(X_test)
                costs.append([costs, classifiersNames[i],seed,split])
                accuracies.append([1-np.sum(np.abs(y_pred-y_test))/len(y_test),classifiersNames[i],seed,split])
    # We could then produce some graphs and means from those results
    return costs, accuracies
    